In [1]:
%run Dataset_cleanup/cleanup_clinical.py

Clinical data set imported


 The following features do not provide any information: 
 ['Composite.Element.REF' 'ethnicity' 'gender' 'pathologicstage'
 'pathologyMstage' 'tumortissuesite'] 



 Variables that are not known at initial diagnosis: 
 ['daystodeath' 'daystolastfollowup' 'daystopsa' 'gleasonscore'
 'histologicaltype' 'numberoflymphnodes' 'pathologyTstage'
 'radiationtherapy' 'residualtumor' 'vitalstatus'] 


Variables that are known at the time of diagnosis:
 ['dateofinitialpathologicdiagnosis' 'psavalue' 'race' 'yearstobirth']

Dimensions of clinical dataframe: (499, 7)


In [2]:
%run Dataset_cleanup/cleanup_gene_counts.py

Gene Counts data set imported

Dimension of DataFrame: (550, 20501)

Transformation Successful!

Transcript abundance estimates have been transformed to transcripts per million reads


## Benchmark Analysis of initial clinical presentation for  predictors of metastasis

In [3]:
"""The Clinical data (including metastasis label) have a multitude of missing items."""
print("Total observations in original dataset:",clinical.shape[0])

not_labeled = y[y.isnull()]
y_labels = y[y.notnull()]

print("\nLabeled observations:",y_labels.shape[0],"\nUnlabeled observations removed:",not_labeled.shape[0])


Total observations in original dataset: 499

Labeled observations: 426 
Unlabeled observations removed: 73


In [4]:
clinical = clinical.loc[y_labels.index]  #filter only observations where metastasis state is known
print("Filtering out missing metastasis state labels left",clinical.shape[0],"observations")

clinical.dropna(inplace = True) # Remove all observations where clinical data is missing
print("Removing NaN rows leaves ",clinical.shape[0]," observations in the feature set.") 

y_labels = y_labels.loc[clinical.index] #remove metastasis labels where no clinical data now exists due to NaN removal.

Filtering out missing metastasis state labels left 426 observations
Removing NaN rows leaves  359  observations in the feature set.


In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer, accuracy_score

In [6]:
recaller = make_scorer(recall_score, greater_is_better=True, needs_proba=False, needs_threshold=False, pos_label='n1')
accuracy = make_scorer(accuracy_score, greater_is_better=True, needs_proba=False, needs_threshold=False)
clf = DecisionTreeClassifier(criterion='gini',
                            splitter='best',
                            max_depth=None,
                            min_samples_split=30,
                            min_samples_leaf=1,
                            min_weight_fraction_leaf=0.0,
                            max_features=None,
                            random_state=None,
                            max_leaf_nodes=None,
                            class_weight='balanced', #avoids the classifier being rewarded for choosing the most prevelant class each instance (n0)
                            presort=False)
recall_vals = cross_val_score(clf, clinical, y_labels, scoring=recaller, cv=5)
accuracy_vals = cross_val_score(clf, clinical, y_labels, scoring=accuracy, cv=5)
print('The recall value for metastasis detection using a simple Decision Tree on patient information that is known at the onset of diagnosis is:')
print(recall_vals.mean())
print('The accuracy score for metasasis vs no metastasis is:\n',accuracy_vals.mean())
print("\nThe null error rate for metastasis detection on this dataset is:")
print((sum(y_labels == 'n1') / len(y_labels)))

The recall value for metastasis detection using a simple Decision Tree on patient information that is known at the onset of diagnosis is:
0.465384615385
The accuracy score for metasasis vs no metastasis is:
 0.530846035115

The null error rate for metastasis detection on this dataset is:
0.16991643454


## Exploratory Analysis using Gene TPM counts as predictors of Metastasis